# 📊 Частотный анализ
Анализируем очищенные тексты:
- Топ редких слов
- Распределение частот
- Аффинитивность слов Вудхауса к корпусу

In [ ]:
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


In [ ]:
AUTHOR_DIR = Path("../data/wodehouse")
CORPUS_DIR = Path("../data/gutenberg")

def load_words(pathlist):
    words = []
    for f in pathlist:
        with open(f, encoding='utf-8') as file:
            words.extend(file.read().lower().split())
    return words

author_words = load_words(AUTHOR_DIR.glob("*.txt"))
print(f"✍️  Слов у автора: {len(author_words):,}")

if CORPUS_DIR.exists():
    corpus_words = load_words(CORPUS_DIR.glob("*.txt"))
    print(f"🌍 Слов в корпусе: {len(corpus_words):,}")
else:
    corpus_words = []
    print("⚠️  Папка с корпусом не найдена. Сравнение будет недоступно.")

In [ ]:
auth_freq = Counter(author_words)
corpus_freq = Counter(corpus_words)

auth_total = sum(auth_freq.values())
corpus_total = sum(corpus_freq.values())

rows = []
for word, count in auth_freq.items():
    if corpus_freq[word] > 0 and count > 5:
        rel_auth = count / auth_total
        rel_corp = corpus_freq[word] / corpus_total
        affinity = rel_auth / rel_corp
        rows.append((word, count, corpus_freq[word], affinity))

aff_df = pd.DataFrame(rows, columns=["word", "author_count", "corpus_count", "affinity"])
aff_df = aff_df.sort_values("affinity", ascending=False)
aff_df.head(10)

In [ ]:
plt.figure(figsize=(12, 5))
sns.histplot(np.log10(aff_df["affinity"] + 1e-9), bins=50)
plt.title("Распределение логарифма аффинитивности")
plt.xlabel("log10(Affinity)")
plt.ylabel("Количество слов")
plt.grid(True)
plt.show()

In [ ]:
aff_df.to_csv("../data/wodehouse_affinity.csv", index=False)
print("✅ Сохранено в data/wodehouse_affinity.csv")